# Proportional hazards with non-linear covariate

In [2]:
from models import *
from data_simulation import *
from score_functions import *

## Exponential

First, we look at how the performance for exponential predictor.

In [9]:
model_scores = pd.DataFrame({
    "model": [],
    "obs": [],
    "score": []
})

model_scores.to_csv("exponential_sim.csv")

model_scores

,model,obs,score


In [74]:
# model_scores = pd.read_csv("exponential_sim.csv",index_col = 0)

,model,obs,score


In [14]:
n_covariates = 1
censor_prop = 0.7
phi = np.exp

for obs in [750]:
    for reps in range(0,100-5-6-43-42):
        print(obs,reps)
        model_scores = pd.read_csv("exponential_sim.csv",index_col = 0)
        
        #sample
        sample = weibull_nonlinear_sim(phi, norm.rvs(size = obs)*2, obs, censor_prop)
        cases, subcohort, cohort, test = cch_splitter(sample)
        
        # unweighted Cox model
        cph1 = fit_cox(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Unweighted Cox PH",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph1,lifelines = True)},
            ignore_index = True
        )
        
        # Cox model with Barlow weights
        cph2 = fit_cox_barlow(cases, subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Cox PH, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph2,lifelines = True)},
            ignore_index = True
        )
        
        cph3 = fit_cox_prentice(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Prentice weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph3,lifelines = True)},
            ignore_index = True
        )
        
        cph4 = fit_cox_self_prentice(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Self-Prentice weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph4,lifelines = True)},
            ignore_index = True
        )
        
        cph5 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Lasso, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph5,lifelines = True)},
            ignore_index = True
        )
        
        cph6 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 1, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Ridge, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph6,lifelines = True)},
            ignore_index = True
        )
        
        cph7 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0.5, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Elastic Net, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph7,lifelines = True)},
            ignore_index = True
        )
        
        tree = unweighted_tree(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Survival Tree, unweighted",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree)},
            ignore_index = True
        )
    
        
        tree1 = ros_tree(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Survival Tree, random oversampler",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree1)},
            ignore_index = True
        )
        
        tree2 = smote_tree(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Survival Tree, SMOTENC",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree2)},
            ignore_index = True
        )
        
        rsf = unweighted_rsf(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, unweighted",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf)},
            ignore_index = True
        )
        
        rsf1 = ros_rsf(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, random oversampler",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf1)},
            ignore_index = True
        )
        
        rsf2 = smote_rsf(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, SMOTENC",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf2)},
            ignore_index = True
        )
        model_scores.to_csv("exponential_sim.csv")
      

750 0
750 1
750 2
750 3


In [4]:
model_scores = pd.DataFrame({
    "model": [],
    "obs": [],
    "score": []
})
model_scores.to_csv("exponential_simulation_brier.csv")
model_scores

# model_scores = pd.read_csv("simple_linear_simulation.csv",index_col = 0)

,model,obs,score


In [5]:
# model_scores = pd.DataFrame({
#     "model": [],
#     "obs": [],
#     "score": []
# })

betas = [0.1,1,2,0.1,1,2]
n_covariates = len(betas)
prop_cat = 0.5
censor_prop = 0.7

for obs in [750]:
# for obs in [1500]:
    for reps in range(0,100):
        print(obs,reps)
        model_scores = pd.read_csv("exponential_simulation_brier.csv",index_col = 0)
        
        #sample
        sample = weibull_simple_linear_sim(betas, prop_cat , obs, censor_prop, pi = 0.5)
        cases, subcohort, cohort, test = cch_splitter(sample)
        
        # unweighted Cox model
        cph1 = fit_cox(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Unweighted Cox PH",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph1,lifelines = True)},
            ignore_index = True
        )
        
        # Cox model with Barlow weights
        cph2 = fit_cox_barlow(cases, subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Cox PH, Barlow weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph2,lifelines = True)},
            ignore_index = True
        )
        
        cph3 = fit_cox_prentice(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Prentice weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph3,lifelines = True)},
            ignore_index = True
        )
        
        cph4 = fit_cox_self_prentice(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Self-Prentice weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph4,lifelines = True)},
            ignore_index = True
        )
        
        cph5 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Lasso, Barlow weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph5,lifelines = True)},
            ignore_index = True
        )
        
        cph6 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 1, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Ridge, Barlow weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph6,lifelines = True)},
            ignore_index = True
        )
        
        cph7 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0.5, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Elastic Net, Barlow weights",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,cph7,lifelines = True)},
            ignore_index = True
        )
        
        tree = unweighted_tree(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Survival Tree, unweighted",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,tree,lifelines = False)},
            ignore_index = True
        )
    
        
        tree1 = ros_tree(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Survival Tree, random oversampler",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,tree1,lifelines = False)},
            ignore_index = True
        )
        
        tree2 = smotenc_tree(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Survival Tree, SMOTENC",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,tree2,lifelines = False)},
            ignore_index = True
        )
        
        rsf = unweighted_rsf(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, unweighted",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,rsf,lifelines = False)},
            ignore_index = True
        )
        
        rsf1 = ros_rsf(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, random oversampler",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,rsf1,lifelines = False)},
            ignore_index = True
        )
        
        rsf2 = smotenc_rsf(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, SMOTENC",
            "obs": obs,
            "score": int_brier_score(cases,subcohort,cohort,test,n_covariates,rsf2,lifelines = False)},
            ignore_index = True
        )
        
        model_scores.to_csv("exponential_simulation_brier.csv")

        
        

750 0
750 1
750 2
750 3
750 4
750 5
750 6
750 7
750 8
750 9
750 10
750 11
750 12
750 13
750 14
750 15
750 16
750 17
750 18
750 19
750 20
750 21
750 22
750 23
750 24
750 25
750 26
750 27
750 28
750 29
750 30
750 31
750 32
750 33
750 34
750 35
750 36
750 37
750 38
750 39
750 40
750 41
750 42
750 43
750 44
750 45
750 46
750 47
750 48
750 49
750 50
750 51
750 52
750 53
750 54
750 55
750 56
750 57
750 58
750 59
750 60
750 61
750 62
750 63
750 64
750 65
750 66
750 67
750 68
750 69
750 70
750 71
750 72
750 73
750 74
750 75
750 76
750 77
750 78
750 79
750 80
750 81
750 82
750 83
750 84
750 85
750 86
750 87
750 88
750 89
750 90
750 91
750 92
750 93
750 94
750 95
750 96
750 97
750 98
750 99


## Quadratic

First, we look at how the performance for exponential predictor.

In [9]:
model_scores = pd.DataFrame({
    "model": [],
    "obs": [],
    "score": []
})
model_scores.to_csv("quadratic.csv")
model_scores

# model_scores = pd.read_csv("exponential_sim.csv",index_col = 0)

,model,obs,score


In [3]:
n_covariates = 1
censor_prop = 0.7
phi = np.exp

for obs in [750]:
    for reps in range(0,100-93-6):
        print(obs,reps)
        model_scores = pd.read_csv("quadratic.csv")
        
        #sample
        sample = weibull_nonlinear_sim(phi, (0.5*norm.rvs(size = obs))**2, obs, censor_prop)
        cases, subcohort, cohort, test = cch_splitter(sample)
        
        # unweighted Cox model
        cph1 = fit_cox(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Unweighted Cox PH",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph1,lifelines = True)},
            ignore_index = True
        )
        
        # Cox model with Barlow weights
        cph2 = fit_cox_barlow(cases, subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Cox PH, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph2,lifelines = True)},
            ignore_index = True
        )
        
        cph3 = fit_cox_prentice(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Prentice weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph3,lifelines = True)},
            ignore_index = True
        )
        
        cph4 = fit_cox_self_prentice(cases, subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Cox PH, Self-Prentice weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph4,lifelines = True)},
            ignore_index = True
        )
        
        cph5 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Lasso, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph5,lifelines = True)},
            ignore_index = True
        )
        
        cph6 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 1, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Ridge, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph6,lifelines = True)},
            ignore_index = True
        )
        
        cph7 = fit_pen_cox_barlow(cases, subcohort,n_covariates, obs, l1_ratio = 0.5, penalizer_show = False)
        model_scores = model_scores.append({
            "model": "Cox PH Elastic Net, Barlow weights",
            "obs": obs,
            "score": concordance_score(n_covariates,test,cph7,lifelines = True)},
            ignore_index = True
        )
        
        tree = unweighted_tree(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Survival Tree, unweighted",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree)},
            ignore_index = True
        )
    
        
        tree1 = ros_tree(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Survival Tree, random oversampler",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree1)},
            ignore_index = True
        )
        
        tree2 = smote_tree(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Survival Tree, SMOTENC",
            "obs": obs,
            "score": concordance_score(n_covariates,test,tree2)},
            ignore_index = True
        )
        
        rsf = unweighted_rsf(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, unweighted",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf)},
            ignore_index = True
        )
        
        rsf1 = ros_rsf(cases,subcohort,n_covariates,obs)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, random oversampler",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf1)},
            ignore_index = True
        )
        
        rsf2 = smote_rsf(cases,subcohort,n_covariates)
        model_scores = model_scores.append({
            "model": "Random Survival Forest, SMOTENC",
            "obs": obs,
            "score": concordance_score(n_covariates,test,rsf2)},
            ignore_index = True
        )
        model_scores.to_csv("quadratic.csv")
      

750 0
